# Glacier DEM Analysis Workflow
This notebook replaces an ArcPy workflow with fully open-source Python tools to:
- Clip DEMs to glacier polygons
- Compute elevation change (difference) rasters
- Perform zonal statistics
- Calculate glacier mass balance
- Visualize difference rasters

👉 Run each cell section-by-section and customize paths to your local data.

In [31]:
import os
import numpy as np
import geopandas as gpd
import rasterio
import rioxarray as rxr
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
print("Modules imported")

Modules imported


In [36]:
# SET YOUR OWN PATHS BELOW
raster_folder = "/Users/milliespencer/Desktop/70_Years_MB Nevados/v1_coregistered_dems" #here's where you store your DEMs
polygon_clips = "example_data_Nevados/Nevados_glacier_shapefiles/Nevados_shapefile_DGA2019/Nevados_polygons_DGA2019.shp" #here's where you store your glacier polygons
stats_field = "COD_GLA" # specify which attribute to reference during the analysis, typically either glacier ID (COD_GLA) or area (AREA_KM2)
snap_raster_path = os.path.join(raster_folder, "CerroBlanco_projected_raster_SRTM_projected_raster_nuth_x+9.32_y+8.27_z+6.54_align.tif") # pick which DEM will be your snap raster (to which your other DEMs will be aligned)
baseline_raster = snap_raster_path
output_folder = "/Users/milliespencer/Desktop/70_Years_MB Nevados/python_workflow_outputs"

In [40]:
raster_path = "/Users/milliespencer/Desktop/70_Years_MB Nevados/v1_coregistered_dems/CerroBlanco_projected_raster_SRTM_projected_raster_nuth_x+9.32_y+8.27_z+6.54_align.tif"
with rasterio.open(raster_path) as src:
    data = src.read(1)  # Read the first band as a NumPy array

## 1. Clip Rasters to Glacier Polygons

In [41]:
# Load the glacier polygons
glacier_polygons = gpd.read_file(polygon_clips)

# Loop over all rasters in the folder
for filename in os.listdir(raster_folder):
    if filename.endswith('.tif') and not filename.endswith('clp.tif'):
        full_path = os.path.join(raster_folder, filename)
        print(f"Clipping: {filename}")
        
        # Open the raster and get its CRS
        raster = rxr.open_rasterio(full_path, masked=True).squeeze()
        raster_crs = raster.rio.crs
        
        # Reproject glacier polygons to the raster's CRS
        glacier_polygons_reprojected = glacier_polygons.to_crs(raster_crs)

        # Clip the raster
        clipped = raster.rio.clip(glacier_polygons_reprojected.geometry, glacier_polygons_reprojected.crs, drop=True)

        # Save the clipped raster
        output_name = os.path.join(output_folder, f"{filename[:20]}_clp.tif")
        clipped.rio.to_raster(output_name)
        print(f"Saved clipped raster: {output_name}")

Clipping: IGM_projected_raster_SRTM_projected_raster_nuth_x-35.24_y-7.77_z-10.15_align.tif
Saved clipped raster: /Users/milliespencer/Desktop/70_Years_MB Nevados/python_workflow_outputs/IGM_projected_raster_clp.tif
Clipping: s37_w072_1arc_v3.tif


/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 28 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 29 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 30 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 31 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 32 will not be rasterized.
  

Saved clipped raster: /Users/milliespencer/Desktop/70_Years_MB Nevados/python_workflow_outputs/s37_w072_1arc_v3.tif_clp.tif
Clipping: LasTermas_projected_raster_SRTM_projected_raster_nuth_x-4.08_y+9.64_z-24.32_align.tif
Saved clipped raster: /Users/milliespencer/Desktop/70_Years_MB Nevados/python_workflow_outputs/LasTermas_projected__clp.tif
Clipping: CerroBlanco_projected_raster_SRTM_projected_raster_nuth_x+9.32_y+8.27_z+6.54_align.tif
Saved clipped raster: /Users/milliespencer/Desktop/70_Years_MB Nevados/python_workflow_outputs/CerroBlanco_projecte_clp.tif


/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 28 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 29 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 30 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 31 will not be rasterized.
  warnings.warn(
/Users/milliespencer/miniconda3/envs/glacier-env/lib/python3.10/site-packages/rasterio/features.py:336: ShapeSkipWarning: Invalid or empty shape None at index 32 will not be rasterized.
  

## 2. Compute Raster Differences vs. Baseline

In [ ]:
baseline = rxr.open_rasterio(baseline_raster, masked=True).squeeze()

for filename in os.listdir(output_folder):
    if filename.endswith('clp.tif'):
        full_path = os.path.join(output_folder, filename)

        if os.path.basename(baseline_raster)[:20] in filename:
            continue

        try:
            raster = rxr.open_rasterio(full_path, masked=True).squeeze()
            difference = raster - baseline

            diff_path = os.path.join(output_folder, filename.replace('_clp.tif', '_diff.tif'))
            difference.rio.to_raster(diff_path)
            print(f"Saved difference raster: {diff_path}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")


## 3. Zonal Statistics and Mass Balance

In [ ]:
for filename in os.listdir(output_folder):
    if filename.endswith('diff.tif'):
        raster_path = os.path.join(output_folder, filename)

        stats = zonal_stats(
            polygon_clips,
            raster_path,
            stats=["mean", "std", "min", "max", "count", "sum"],
            geojson_out=True,
            nodata=None
        )

        stats_gdf = gpd.GeoDataFrame.from_features(stats)
        stats_gdf['area_m2'] = stats_gdf['geometry'].area * 1e6
        stats_gdf['mass_balance_mwe'] = (858 * stats_gdf['mean']) / (1000 * stats_gdf['area_m2'])

        output_csv = os.path.join(output_folder, filename.replace('.tif', '_zonalStats.csv'))
        stats_gdf[[stats_field, 'mean', 'std', 'min', 'max', 'count', 'sum', 'area_m2', 'mass_balance_mwe']].to_csv(output_csv, index=False)
        print(f"Saved zonal stats: {output_csv}")


## 4. Visualize Difference Rasters

In [ ]:
for filename in os.listdir(output_folder):
    if filename.endswith('diff.tif'):
        map_path = os.path.join(output_folder, filename)
        with rasterio.open(map_path) as src:
            raster_data = src.read(1)
            extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

        plt.figure(figsize=(8, 6))
        plt.imshow(raster_data, cmap='viridis', extent=extent)
        plt.colorbar(label='Elevation Change (m)')
        plt.title(f"Raster Difference: {filename}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()
